In [1]:
import aiida

# must execute it in the first place
aiida.load_profile()


In [2]:
from aiida.orm import Code
from aiida.orm import Str, Int, Dict, List, Float
from aiida.engine import calcfunction, WorkChain, ToContext, append_
from aiida.plugins import DataFactory, WorkflowFactory
from itertools import cycle
import os

from aiida.engine import calcfunction, workfunction, submit, run
from aiida.orm import load_code, load_node
from tools import wait_for_node_finished


In [3]:
# codeの定義を行う。
from tools.aiida_support import get_or_create_local_computer, get_or_create_code
from os.path import expanduser
import os
home = expanduser("~")
work_directory = os.path.join(home, 'aiida')
computer_local = get_or_create_local_computer(work_directory, 'localhost')
code = get_or_create_code('alamode.alm_suggest',
                          computer_local,
                          '/home/max/Documents/alamode/bin/alm')
code_alm_sugget = code

code_alm_sugget


<Code: Remote code 'alamode.alm_suggest' on localhost, pk: 2326, uuid: 32253ce4-318f-4c70-b09d-977a5307c78b>

In [4]:
import json
with open("setting.json") as f:
    _dic = json.load(f)
print(_dic)
CWD = _dic["CWD"]
_path = os.path.abspath(CWD)
os.makedirs(_path, exist_ok=True)
_path


{'CWD': 'run35'}


'/home/max/Documents/aiida-alamode_public/example/run35'

In [5]:
from os.path import expanduser
from tools import NodeBank


# 再実行時に作成したノードを用いるためにnodebankを使う。
g_force = False
nodebank = NodeBank(_path, force=g_force)


In [6]:

g_superstructure = nodebank.load("superstructure")  # 長周期安定構造

g_norder = nodebank.dump('cubic_norder', Int(2))  # cubic term
g_prefix = nodebank.dump('cubic_prefix', Str("cubic"))  # cubic term prefix
g_norder, g_prefix


(<Int: uuid: 79b9b510-e505-4397-8faf-21ded8dac26f (pk: 3693) value: 2>,
 <Str: uuid: 365abe8c-3a31-4aa7-b7cf-5da1f911303b (pk: 3694) value: cubic>)

In [7]:
g_cutoff = Dict(dict={"*-*": {None, 7.3}})  # cubic term cutoff 定義

g_cutoff = nodebank.load_or_dump("cutoff2", g_cutoff)
g_cutoff


debug, node uuid: f7c13dfc-c9f9-4844-9386-8e9824af86fa (unstored)
debug, load None
debug, force dump


<Dict: uuid: f7c13dfc-c9f9-4844-9386-8e9824af86fa (pk: 3695)>

In [8]:
g_standardizedstructure = nodebank.load("standardizedstructure")

g_diag = nodebank.load("factor")  # 長周期
g_almprefix = "_".join([g_standardizedstructure.get_formula(),
                        "x".join(map(str, g_diag.get_array("factor").reshape(-1))),
                        g_prefix.value,
                        ])
print(g_almprefix)

g_wd_dos = Str(g_almprefix)
g_wd_dos


Si8_2x0x0x0x2x0x0x0x2_cubic


<Str: uuid: 04e3c012-4fac-4bf1-8afe-a891ed529de3 (unstored) value: Si8_2x0x0x0x2x0x0x0x2_cubic>

In [9]:
g_root_wd = nodebank.load('root_wd')  # root directory
g_root_wd
g_cwd = Str(os.path.join(g_root_wd.value, g_wd_dos.value))
print(g_cwd)
g_cwd = nodebank.dump(f'{g_prefix.value}_cwd', g_cwd)  # cubic termの結果を置くsubdirectory
os.makedirs(g_cwd.value, exist_ok=True)


uuid: edaf2272-df44-4202-816b-4e3d50f91415 (unstored) value: /home/max/Documents/aiida-alamode_public/example/run35/Si8_2x0x0x0x2x0x0x0x2_cubic


In [10]:

code = code_alm_sugget

builder = code.get_builder()
builder.structure = g_superstructure  # 長周期構造
builder.prefix = g_prefix  # 名前付け
builder.cwd = Str(os.path.join(g_cwd.value, f"{g_prefix.value}_alm_suggest"))  # 　保存directory
builder.norder = g_norder
builder.cutoff = g_cutoff

builder.metadata = {
    'options': {
        'resources': {'tot_num_mpiprocs': 1, 'num_machines': 1},
        'input_filename': "alm_suggest_cubic.in",
        'output_filename': "alm_suggest_cubic.out",
    }}

key = f"{g_prefix.value}_alm_suggest"

g_alm_suggest_future = nodebank.load_code_or_wait_for_node(key, builder, sec=5)


None
uuid: 31511078-cb21-4d32-a959-6eea4a158cd6 (pk: 3700) (aiida.calculations:alamode.alm_suggest)
wait another 5 sec.
wait another 5 sec.
wait another 5 sec.
True True
0:00:15.319097


In [11]:
nodebank.dump(f"{g_prefix.value}_suggest_pattern", g_alm_suggest_future.outputs.pattern)
nodebank.dump(f"{g_prefix.value}_pattern",
              g_alm_suggest_future.outputs.pattern)
nodebank.dump(f"{g_prefix.value}_suggest_result", g_alm_suggest_future.outputs.results)


<Dict: uuid: f5fc5d7d-06d3-4500-aa1e-7684cc5b2164 (pk: 3703)>